In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import csv
import os
import random
import math

from torch import nn

In [ ]:
TARGET_COLUMN = 'target'
METAINFO_COLUMNS = ['stage', 'move_count', 'game_index', 'weight']
METAINFO_COLUMNS_COUNT = len(METAINFO_COLUMNS)
STAGE_INDEX = 0
MOVE_COUNT_INDEX = 1
WEIGHT_INDEX = 3
TARGET_INDEX = -1

STARTPOS_STAGE = 24
NUMBER_OF_FEATURES = 52

TAPERED_EVAL_BOUND = 6 + NUMBER_OF_FEATURES + 64 * 6

NUMBER_OF_GAMES = 100000
BATCH_SIZE = 32768

In [ ]:
move_number_weights = [0.00001, 0.00008, 0.00024, 0.00043, 0.00057, 0.0007, 0.00085, 0.00105, 0.00139, 0.00181, 0.00244, 0.00375, 0.00521, 0.00731, 0.00973, 0.01254, 0.01645, 0.02553, 0.03776, 0.05396, 0.07285, 0.09648, 0.12129, 0.15135, 0.18169, 0.21407, 0.245858, 0.277823, 0.309948, 0.341158, 0.37258, 0.402667, 0.432192, 0.45969, 0.487255, 0.514445, 0.539746, 0.564126, 0.587609, 0.609724, 0.629945, 0.650371, 0.668892, 0.687291, 0.704763, 0.720856, 0.735872, 0.750076, 0.763791, 0.776711, 0.78944, 0.801965, 0.814003, 0.824748, 0.83541, 0.844756, 0.853865, 0.862123, 0.870204, 0.878105, 0.885155, 0.891836, 0.898314, 0.90489, 0.91067, 0.915671, 0.920825, 0.925534, 0.930256, 0.934641, 0.93932, 0.943253, 0.947136, 0.950907, 0.954602, 0.957961, 0.961024, 0.963216, 0.965767, 0.968335, 0.970546, 0.972861, 0.974728, 0.976481, 0.978039, 0.979738, 0.981401, 0.98274, 0.984218, 0.985646, 0.986751, 0.987752, 0.988783, 0.989843, 0.990658, 0.991329, 0.99217, 0.992891, 0.993448, 0.99403]
stage_weights = [0.864812, 0.862839, 1.5417, 2.14211, 0.637247]

class Row:
    def __init__(self, csv_line):
        self.X = np.array(csv_line[METAINFO_COLUMNS_COUNT:TARGET_INDEX], dtype=np.single)
        self.y = csv_line[TARGET_INDEX]
        move_number_weight = 1
        stage_weight = stage_weights[math.floor(csv_line[STAGE_INDEX] + 0.25) // 5]
        if csv_line[MOVE_COUNT_INDEX] <= 100:
            move_number_weight = move_number_weights[math.floor(csv_line[MOVE_COUNT_INDEX] + 0.25) - 1] * 100000 / NUMBER_OF_GAMES
        # self.w = move_number_weight * stage_weight
        self.w = 1
        self.X = np.concatenate((self.X, self.X))
        self.X[:TAPERED_EVAL_BOUND] *= csv_line[STAGE_INDEX] / 24
        self.X[TAPERED_EVAL_BOUND:] *= 1 - csv_line[STAGE_INDEX] / 24
        
    def get_not_features_count():
        return METAINFO_COLUMNS_COUNT + 1
    
    def get_X(self):
        return self.X
    
    def get_y(self):
        return self.y
    
    def get_w(self):
        return self.w

In [ ]:
class StatCalcer:
    def __init__(self, row_size):
        self.max = np.zeros(row_size, dtype=np.single)
        self.min = np.full(row_size, 2 ** 16, dtype=np.single)
        self.cnt = 0

    def process_row(self, row):
        self.max = np.maximum(self.max, row.get_X())
        self.min = np.minimum(self.min, row.get_X())
        self.cnt += 1

    def get_max(self):
        return self.max
    
    def get_min(self):
        return self.min
    
    def get_cnt(self):
        return self.cnt


In [ ]:
class DistributedDatasetHolder:
    def __init__(self, dataset_dir, sample_test_rate):
        self.dataset_dir = dataset_dir
        self.dataset_chunks = [entry for entry in os.listdir(dataset_dir) if os.path.isfile(os.path.join(self.dataset_dir, entry))]
        self.n = len(self.dataset_chunks)
        
        self.train_chunks_count = math.floor(len(self.dataset_chunks) * sample_test_rate + 0.49)
        print(self.train_chunks_count)
        self.dataset_chunks = sorted(self.dataset_chunks)
        print(self.dataset_chunks)

        self.open_files()

        self.shuffle()
        
        self.stat_calcer = StatCalcer(self.get_features_count())
        self.calc_stat()
    
    def open_files(self):
        self.chunk_files = [open(os.path.join(self.dataset_dir, filename)) for filename in self.dataset_chunks]
        self.csv_readers = [csv.reader(file) for file in self.chunk_files]
        for i in range(self.n):
            self.header = np.array(next(self.csv_readers[i]))

    def close_files(self):
        for file in self.chunk_files:
            file.close()

    def reopen_files(self):
        self.close_files()
        self.open_files()

    def calc_stat(self):
        print("calc")
        self.reopen_files()
        for i in range(self.n):
            while True:
                try:
                    row = Row(np.array(next(self.csv_readers[i]), dtype=np.single))
                except StopIteration:
                    break
                except Exception:
                    raise RuntimeError("Stopped!")
                self.stat_calcer.process_row(row)
        self.reopen_files()
        print(self.stat_calcer.get_cnt())

    def shuffle_file(self, filename):
        with open(os.path.join(self.dataset_dir, filename)) as file:
            lines = np.array(file.readlines())
            np.random.shuffle(lines[1:])
        with open(os.path.join(self.dataset_dir, filename), 'w') as file:
            file.writelines(lines)
        
    def shuffle(self):
        print("shuffle")
        for filename in self.dataset_chunks:
            self.shuffle_file(filename)
        self.reopen_files()

    def get_batch(self, left_pos, right_pos, batch_size, norm):
        batch_X = np.zeros((batch_size, self.get_features_count()), dtype=np.single)
        batch_y = np.zeros(batch_size, dtype=np.single)
        batch_w = np.zeros(batch_size, dtype=np.single)
        size = 0
        while size < batch_size:
            pos = random.randint(left_pos, right_pos - 1)
            try:
                row = Row(np.array(next(self.csv_readers[pos]), dtype=np.single))
            except StopIteration:
                self.shuffle()
                return self.get_batch(left_pos, right_pos, batch_size, norm)
            except Exception:
                raise RuntimeError("Stopped!")
            batch_X[size, :] = row.get_X() / norm
            batch_y[size] = row.get_y()
            batch_w[size] = row.get_w()
            size += 1
        return batch_X, batch_y, batch_w

    def get_train_batch(self, batch_size, norm):
        return self.get_batch(0, self.train_chunks_count, batch_size, norm)
    
    def get_test_batch(self, batch_size, norm):
        return self.get_batch(self.train_chunks_count, self.n, batch_size, norm)
    
    def get_features_count(self):
        return 64 * 12 + 6 * 2 + NUMBER_OF_FEATURES * 2
    
    def get_stat_calcer(self):
        return self.stat_calcer
    
    def __del__(self):
        self.close_files()

In [ ]:
dataset_holder = DistributedDatasetHolder('/home/wind-eagle/quirky_data/dataset', 0.9)

In [ ]:
stat_calcer = dataset_holder.get_stat_calcer()
X_max = stat_calcer.get_max()
X_min = stat_calcer.get_min()
X_norm = X_max - X_min
X_norm = np.maximum(np.min(X_norm[X_norm > 1e-9]), X_norm)
X_norm

In [ ]:
model = nn.Sequential()
model.add_module('linear', nn.Linear(dataset_holder.get_features_count(), 1, bias=False))
model.add_module('sigmoid', nn.Sigmoid())
opt = torch.optim.Adam(model.parameters(), lr=0.01)

def get_loss(model, X, y, w, C=0.0):
    y_pred = model(X)[:, 0]
    assert y_pred.dim() == 1
    loss = torch.sum(w * ((y - y_pred) ** 2)) / torch.sum(w)
    loss += C * torch.norm(model.linear.weight, 2)
    return loss

def get_weights():
    ww = (model.linear.weight[0]).detach().numpy() / X_norm
    for i in range(5):
        l = 6 + NUMBER_OF_FEATURES + 64 * i
        r = 6 + NUMBER_OF_FEATURES + 64 * (i + 1)
        if i == 0:
            l += 8
            r -= 8
        m = np.median(ww[l:r])
        ww[l:r] -= m
        ww[i] += m

    for i in range(5):
        l = TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * i
        r = TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * (i + 1)
        if i == 0:
            l += 8
            r -= 8
        m = np.median(ww[l:r])
        ww[l:r] -= m
        ww[i + TAPERED_EVAL_BOUND] += m

    ww = (ww / ((ww[0] + ww[TAPERED_EVAL_BOUND]) / 2) * 100).astype(np.int32)
    return ww

In [ ]:
def main_learning(iter_count):
    C = 3e-5
    history = []

    for i in range(iter_count):
        np_X, np_y, np_w = dataset_holder.get_train_batch(BATCH_SIZE, X_norm)
        X_batch = torch.tensor(np_X, dtype=torch.float32)
        y_batch = torch.tensor(np_y, dtype=torch.float32)
        w_batch = torch.tensor(np_w, dtype=torch.float32)

        loss = get_loss(model, X_batch, y_batch, w_batch, C=C)

        loss.backward()
        
        opt.step()
        opt.zero_grad()

        history.append(loss.data.numpy())

        if i % 100 == 0:
            weights = get_weights()
            print(weights[:6])
            print(weights[TAPERED_EVAL_BOUND:TAPERED_EVAL_BOUND + 6])
            print(weights[6 : 6 + NUMBER_OF_FEATURES])
            print(weights[TAPERED_EVAL_BOUND + 6 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES])
        if i % 100 == 0:
            np_X, np_y, np_w = dataset_holder.get_test_batch(BATCH_SIZE, X_norm)
            X_test_batch = torch.tensor(np_X, dtype=torch.float32)
            y_test_batch = torch.tensor(np_y, dtype=torch.float32)
            w_test_batch = torch.tensor(np_w, dtype=torch.float32)
            test_loss = get_loss(model,
                torch.tensor(X_test_batch, dtype=torch.float32),
                torch.tensor(y_test_batch, dtype=torch.float32),
                torch.tensor(w_test_batch, dtype=torch.float32),
                C=0.0).detach().numpy().sum()
            train_loss = np.mean(history[-40:])
            print(f"step #{i}, train_loss = {train_loss:.4f}, test_loss = {test_loss:.4f}")
        elif i % 10 == 0:
            train_loss = np.mean(history[-40:])
            print(f"step #{i}, train_loss = {train_loss:.4f}")
        

main_learning(7500)

In [ ]:
weights = get_weights()

In [ ]:
weights[:6]

In [ ]:
weights[TAPERED_EVAL_BOUND:TAPERED_EVAL_BOUND + 6]

In [ ]:
weights[6 : NUMBER_OF_FEATURES + 6]

In [ ]:
weights[TAPERED_EVAL_BOUND + 6 : TAPERED_EVAL_BOUND + NUMBER_OF_FEATURES + 6]

In [ ]:
def display_mat(mat):
    plt.matshow(mat)
    for (x, y), value in np.ndenumerate(mat):
        plt.text(y, x, f"{value}", va="center", ha="center")

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 0 : 6 + NUMBER_OF_FEATURES + 64 * 1].reshape(8, 8))

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 1 : 6 + NUMBER_OF_FEATURES + 64 * 2].reshape(8, 8))

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 2 : 6 + NUMBER_OF_FEATURES + 64 * 3].reshape(8, 8))

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 3 : 6 + NUMBER_OF_FEATURES + 64 * 4].reshape(8, 8))

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 4 : 6 + NUMBER_OF_FEATURES + 64 * 5].reshape(8, 8))

In [ ]:
display_mat(weights[6 + NUMBER_OF_FEATURES + 64 * 5 : 6 + NUMBER_OF_FEATURES + 64 * 6].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 0 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 1].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 1 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 2].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 2 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 3].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 3 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 4].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 4 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 5].reshape(8, 8))

In [ ]:
display_mat(weights[TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 5 : TAPERED_EVAL_BOUND + 6 + NUMBER_OF_FEATURES + 64 * 6].reshape(8, 8))

In [ ]:
weights_of_psq = weights[6 + NUMBER_OF_FEATURES:]
psq = [(0, 0)] * (64 * 6)
for i in range(6):
    for j in range(64):
        psq[i * 64 + j] = (weights_of_psq[i * 64 + j], weights_of_psq[TAPERED_EVAL_BOUND + i * 64 + j])

for i in range(len(psq)):
    print(f"ScorePair({psq[i][0]}, {psq[i][1]}),", end="")
    if i % 8 == 7:
        print()

In [ ]:
weights_of_features = weights[6:]
features = [(0, 0)] * (NUMBER_OF_FEATURES)
for i in range(NUMBER_OF_FEATURES):
    features[i] = (weights_of_features[i], weights_of_features[TAPERED_EVAL_BOUND + i])

for i in range(NUMBER_OF_FEATURES):
    print(f"ScorePair({features[i][0]}, {features[i][1]}),", end="")
    if i % 8 == 7:
        print()